<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [5]:
import pandas as pd
import tweepy
import re
import regex
import emoji

In [4]:
api_key="#########"
auth = tweepy.OAuthHandler("P2j2I0zPIOKlFcE6tLXFYrUMs", "103Sp4rD3tDTlOwHvmRgi0RplmN6Mb80oLiIc0sC0qVcW0T28t")
auth.set_access_token("1362577829691609088-tiZkEvLZQn1JjlI7VO15lETDFIDvlK","Fsoz17Fi43fDH76wm33rE01urAhKofXV5Su9DgwfyTUWh" )
api = tweepy.API(auth, wait_on_rate_limit=True)

In [7]:
tweets = tweepy.Cursor(api.search,
                       q=['#fortnite'],
                       lang='es',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)

In [8]:
tweets

In [9]:
t = [tweet for tweet in tweets]

In [11]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    t_processed.append(temp)
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data

,tweets,date,city,user_location,id,link
0,"La sección de ""Personaliza tu héroe"" ha sido a...",2021-05-20 20:18:18,Bogotá,Costa Rica,1395473979935272964,https://twitter.com/i/web/stuatus/139547397993...
1,"El paquete de desafíos de ""Sombras Callejeras""...",2021-05-20 20:18:14,Bogotá,Madrid,1395473965418754051,https://twitter.com/i/web/stuatus/139547396541...
2,RT @Jorge_Most_: Re-Subo el enlace para conseg...,2021-05-20 20:17:18,Bogotá,"Xalapa, Veracruz de Ignacio de",1395473728075702279,https://twitter.com/i/web/stuatus/139547372807...
3,"La sección de ""Personaliza tu Héroe"" ha sido a...",2021-05-20 20:16:36,Bogotá,Madrid,1395473553911435267,https://twitter.com/i/web/stuatus/139547355391...
4,RT @Jorge_Most_: El Pack Sombra Rubí también e...,2021-05-20 20:16:04,Bogotá,,1395473417856552960,https://twitter.com/i/web/stuatus/139547341785...
...,...,...,...,...,...,...
95,RT @Jorge_Most_: Se han activado los Drops de ...,2021-05-20 19:44:05,Bogotá,Argentina,1395465371772203011,https://twitter.com/i/web/stuatus/139546537177...
96,RT @Jorge_Most_: Re-Subo el enlace para conseg...,2021-05-20 19:43:58,Bogotá,"Las Palmas, Islas Canarias",1395465339677429760,https://twitter.com/i/web/stuatus/139546533967...
97,RT @3DJuegosMX: Y hablando de la NBA: la liga ...,2021-05-20 19:43:56,Bogotá,,1395465331120934916,https://twitter.com/i/web/stuatus/139546533112...
98,RT @NotPaloleaks: La sección de ¡Personaliza t...,2021-05-20 19:43:39,Bogotá,"Austintown, Ohio",1395465261478817800,https://twitter.com/i/web/stuatus/139546526147...


In [12]:
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions
  
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

In [13]:
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list
  
data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

,tweets,date,city,user_location,id,link,texto_limpio,links,hashtags,mentions,emojis
0,"La sección de ""Personaliza tu héroe"" ha sido a...",2021-05-20 20:18:18,Bogotá,Costa Rica,1395473979935272964,https://twitter.com/i/web/stuatus/139547397993...,"La sección de ""Personaliza tu héroe"" ha sido a...",[https://t.co/YR9A7XZsdJ],[#Fortnite],[],[]
1,"El paquete de desafíos de ""Sombras Callejeras""...",2021-05-20 20:18:14,Bogotá,Madrid,1395473965418754051,https://twitter.com/i/web/stuatus/139547396541...,"El paquete de desafíos de ""Sombras Callejeras""...",[https://t.co/RrnePHzlGy],[#Fortnite],[],[]
2,RT @Jorge_Most_: Re-Subo el enlace para conseg...,2021-05-20 20:17:18,Bogotá,"Xalapa, Veracruz de Ignacio de",1395473728075702279,https://twitter.com/i/web/stuatus/139547372807...,RT Re-Subo el enlace para conseguir el pack d...,[],[],[@Jorge_Most_:],[]
3,"La sección de ""Personaliza tu Héroe"" ha sido a...",2021-05-20 20:16:36,Bogotá,Madrid,1395473553911435267,https://twitter.com/i/web/stuatus/139547355391...,"La sección de ""Personaliza tu Héroe"" ha sido a...",[https://t.co/UAggxBvFzh],[#Fortnite],[],[]
4,RT @Jorge_Most_: El Pack Sombra Rubí también e...,2021-05-20 20:16:04,Bogotá,,1395473417856552960,https://twitter.com/i/web/stuatus/139547341785...,RT El Pack Sombra Rubí también está disponibl...,[],[#Fortnite],"[@Jorge_Most_:, @Egyptian_Leaker]",[📸]


In [14]:
data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp

,hora,cnt
8,19:43:00,4
16,19:44:00,3
2,19:45:00,5
24,19:46:00,2
15,19:47:00,3
13,19:48:00,3
14,19:49:00,3
34,19:50:00,1
9,19:51:00,4
17,19:52:00,3


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [15]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [16]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)